In [ ]:
import requests

# Set the URL for the API
BASE_URL = "https://dlmm-api.meteora.ag"

# Define the end point
endpoint = "/pair/all_with_pagination"

# Parameters
params = {
    "page": "1",
    "limit": "10",
    "sort_key": "feetvlratio30m",
    "order_by": "desc", 
    "hide_low_tvl": "7000"
}

# Make the request
try:
    response = requests.get(BASE_URL + endpoint, params=params)
    response.raise_for_status() #raise error if it fails
    data = response.json() #parse the json response
    print(data)

except requests.exceptions.RequestException as e:
    print("Request failed: %s" % e)

{'pairs': [{'address': '4CTUHtiHrPHFT4Zc1qNScrposmM7xfupU7EVDWCR7PZw', 'name': 'TRUMP-SOL', 'mint_x': '6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN', 'mint_y': 'So11111111111111111111111111111111111111112', 'reserve_x': '8rpWRkvkhaY51zDG6cj6nc2HqxkuFNsAudnXn1yc5H4e', 'reserve_y': '7CzwaJ6cNY54579S6hP9udGhQYoSZapWWdHj3FrdKX1Y', 'reserve_x_amount': 76595309473, 'reserve_y_amount': 3219254363756, 'bin_step': 125, 'base_fee_percentage': '5', 'max_fee_percentage': '7.6367188', 'protocol_fee_percentage': '5', 'liquidity': '2059057.9851343262', 'reward_mint_x': '11111111111111111111111111111111', 'reward_mint_y': '11111111111111111111111111111111', 'fees_24h': 14188.418746530344, 'today_fees': 9275.880138400831, 'trade_volume_24h': 296779.7461285305, 'cumulative_trade_volume': '740274548.8600', 'cumulative_fee_volume': '42630907.5900', 'current_price': 0.09315081063448666, 'apr': 0.689073297059419, 'apy': 1126.1835524010162, 'farm_apr': 0.0, 'farm_apy': 0.0, 'hide': False, 'is_blacklisted': F

In [1]:
import requests
import pandas as pd

def fetch_all_pairs(
    base_url: str = "https://dlmm-api.meteora.ag",
    include_unknown: bool = True,
    page: int = 0,
    limit: int = 50,
    hide_low_tvl: float = 0.0
):
    """
    Fetches all pairs using the /pair/all_with_pagination endpoint.
    Adjust the query parameters as needed.
    """
    endpoint = f"{base_url}/pair/all_with_pagination"
    params = {
        "include_unknown": str(include_unknown).lower(),
        "page": page,
        "limit": limit,
        "hide_low_tvl": 7000,
        "sort_key": "feetvlratio30m",
        "order_by": "desc"
    }
    resp = requests.get(endpoint, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

# Fetch the data
pairs_data = fetch_all_pairs()

# Keep only a subset of columns
columns_to_keep = [
    'address', 'name', 'mint_x', 'bin_step', 'base_fee_percentage', 'max_fee_percentage',
    'liquidity', 'fees_24h', 'trade_volume_24h', 'fees', 'fee_tvl_ratio', 'volume'
]
df = pd.DataFrame(pairs_data['pairs'])[columns_to_keep]

# Extract 30m and 1h only, discard 2h
for col in ['fees', 'fee_tvl_ratio', 'volume']:
    df[f"{col}_min_30"] = df[col].apply(lambda x: x.get('min_30') if isinstance(x, dict) else None)
    df[f"{col}_hour_1"] = df[col].apply(lambda x: x.get('hour_1') if isinstance(x, dict) else None)

# Drop the original columns
df.drop(columns=['fees', 'fee_tvl_ratio', 'volume'], inplace=True)

# Keep the 30m fee_tvl_ratio in its own column
df['fee_tvl_ratio_30m'] = df['fee_tvl_ratio_min_30']

def simple_momentum(row, col_prefix):
    v30 = row[f"{col_prefix}_min_30"]
    v1h = row[f"{col_prefix}_hour_1"]
    if v30 is None or v1h is None:
        return None
    # If 30m * 2 > 1h => "+", else "-"
    return "+" if (v30 * 2) > v1h else "-"

# Add momentum columns
df['fees_momentum'] = df.apply(lambda row: simple_momentum(row, "fees"), axis=1)
df['fee_tvl_ratio_momentum'] = df.apply(lambda row: simple_momentum(row, "fee_tvl_ratio"), axis=1)
df['volume_momentum'] = df.apply(lambda row: simple_momentum(row, "volume"), axis=1)

# Reorder and rename columns
desired_columns = [
    'name',
    'address',
    'mint_x',
    'liquidity',                # Will become 'TVL'
    'fee_tvl_ratio_min_30',     # Will become 'Fee/TVL 30m'
    'bin_step',
    'base_fee_percentage',      # Will become 'Fee %'
    'max_fee_percentage',       # Will become 'Max fee %'
    'fee_tvl_ratio_momentum',   # Will become 'Fee/TVL mom'
    'volume_momentum',          # Will become 'Vol mom'
    'fees_momentum',            # Will become 'Fees mom'
    'volume_min_30'            # Will become 'Vol 30m'
]
df = df[desired_columns].rename(columns={
    'liquidity': 'TVL',
    'address': 'DLMM Address',
    'mint_x': 'CA',
    'fee_tvl_ratio_min_30': 'Fee/TVL 30m',
    'base_fee_percentage': 'Fee %',
    'max_fee_percentage': 'Max fee %',
    'fee_tvl_ratio_momentum': 'Fee/TVL mom',
    'volume_momentum': 'Vol mom',
    'fees_momentum': 'Fees mom',
    'volume_min_30': 'Vol 30m'
})

# Add a prefix to the DLMM Address column
df["DLMM Address"] = df["DLMM Address"].apply(lambda addr: f"https://app.meteora.ag/dlmm/{addr}")

# Create a new column "APR 30m" after "Fee/TVL 30m"
# First remove any '%' characters
df["Fee %"] = df["Fee %"].astype(str).str.strip('%')

# Now convert to float and divide by 100
df["Fee %"] = pd.to_numeric(df["Fee %"], errors="coerce")

# Then handle Vol 30m and TVL
df["Vol 30m"] = pd.to_numeric(df["Vol 30m"], errors="coerce")
df["TVL"] = pd.to_numeric(df["TVL"], errors="coerce")

df["APR 30m"] = (df["Vol 30m"] * df["Fee %"]) / df["TVL"]

# Reorder columns to place "APR 30m" right after "Fee/TVL 30m"
final_column_order = [
    'name',
    'Fee/TVL 30m',
    'APR 30m',
    'bin_step',
    'Fee %',
    'Max fee %',
    'Fee/TVL mom',
    'Vol mom',
    'Fees mom',
    'Vol 30m',
    'TVL',
    'CA',
    'DLMM Address'
]
df = df[final_column_order]

# Optionally adjust display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', None)  # or a large integer

print(df.head(10))

            name  Fee/TVL 30m    APR 30m  bin_step  Fee %  Max fee % Fee/TVL mom Vol mom Fees mom        Vol 30m           TVL                                            CA                                                              DLMM Address
0  MEGAVERSE-SOL    35.726157  24.919841       125    5.0  7.6367188           +       +        +   80213.030334  16094.209907  54Pu4S2mxrVuhNh913WFLFpxntFG1pnXJ1Bv5C7rpump  https://app.meteora.ag/dlmm/CgUsVderrHvwfpmsKxdL4BhzY2FTZF9itJhcHWhURou7
1       PXNV-SOL    16.468290   9.689240       400    5.0         10           +       +        +   76949.808896  39708.899540  BB6nXS2RZ8QVRzYL9xtK9X6HQd5cUxWNyVoJqXZ3imVW  https://app.meteora.ag/dlmm/Cq9XGMiJ24huewTJCnR7ndXuGQBmXGzj6mArKWfR3abL
2  MEGAVERSE-SOL    16.175543  10.205817       250    5.0         10           +       +        +   29003.195125  14209.149529  54Pu4S2mxrVuhNh913WFLFpxntFG1pnXJ1Bv5C7rpump  https://app.meteora.ag/dlmm/6oJsV2xneorQZbjY5fnvbLf8o6pJqqbi2cy3SaFyvSGU
3       ORBI